# Project

In [109]:
import xgboost as xgb
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, RobustScaler
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.model_selection import GridSearchCV, StratifiedKFold, RandomizedSearchCV

In [110]:
train_df = pd.read_excel('Horse-Data_student.xlsx', sheet_name = 'Training')
test_df = pd.read_excel('Horse-Data_student.xlsx', sheet_name = 'Test')
exam_df = pd.read_excel('Horse-Data_student.xlsx', sheet_name = 'Exam')

In [111]:
train_df.head()

,Date,Raceno,HorseNo,NewH,Session,NoRace,Going,Ncourse,HV,AWT,...,JAveStdRank60,TAveStdRank30,AveSpeedRating,LastSpeedRating,WtCarriedChg,DistTChg,LWP60,Runners,Trainer,Rank
0,2004-09-05,1,3,0,2004,1,2,0.0,0,0,...,0.006270,0.016166,-0.486870,6.440000e-18,13.142857,-0.083685,0.501978,14,CSShum,1
1,2004-09-05,1,11,0,2004,1,2,0.0,0,0,...,-0.094583,0.035210,0.059599,6.440000e-18,-6.857143,-0.276057,0.707772,14,YOWong,2
2,2004-09-05,1,4,0,2004,1,2,0.0,0,0,...,0.001678,-0.048561,0.005756,-1.459295e+00,13.142857,-0.276057,0.752324,14,CFownes,3
3,2004-09-05,1,2,0,2004,1,2,0.0,0,0,...,-0.104668,0.080768,-0.103874,6.440000e-18,12.142857,-0.083685,-0.630849,14,DOughton,4
4,2004-09-05,1,8,0,2004,1,2,0.0,0,0,...,0.037451,0.044965,-0.162778,-4.902859e-01,-6.857143,0.080618,0.877263,14,JMoore,5


In [112]:
# create unique id
def create_unique_id(df):
    df["Race_Horse_ID"] = df["Date"].astype(str) + '_' + df["Raceno"].astype(str) + '_' + df["HorseNo"].astype(str)
    df["Race_ID"] = df["Date"].astype(str) + '_' + df["Raceno"].astype(str)
    return df

train_df = create_unique_id(train_df)
test_df = create_unique_id(test_df)
exam_df = create_unique_id(exam_df)
train_df.head()

,Date,Raceno,HorseNo,NewH,Session,NoRace,Going,Ncourse,HV,AWT,...,AveSpeedRating,LastSpeedRating,WtCarriedChg,DistTChg,LWP60,Runners,Trainer,Rank,Race_Horse_ID,Race_ID
0,2004-09-05,1,3,0,2004,1,2,0.0,0,0,...,-0.486870,6.440000e-18,13.142857,-0.083685,0.501978,14,CSShum,1,2004-09-05_1_3,2004-09-05_1
1,2004-09-05,1,11,0,2004,1,2,0.0,0,0,...,0.059599,6.440000e-18,-6.857143,-0.276057,0.707772,14,YOWong,2,2004-09-05_1_11,2004-09-05_1
2,2004-09-05,1,4,0,2004,1,2,0.0,0,0,...,0.005756,-1.459295e+00,13.142857,-0.276057,0.752324,14,CFownes,3,2004-09-05_1_4,2004-09-05_1
3,2004-09-05,1,2,0,2004,1,2,0.0,0,0,...,-0.103874,6.440000e-18,12.142857,-0.083685,-0.630849,14,DOughton,4,2004-09-05_1_2,2004-09-05_1
4,2004-09-05,1,8,0,2004,1,2,0.0,0,0,...,-0.162778,-4.902859e-01,-6.857143,0.080618,0.877263,14,JMoore,5,2004-09-05_1_8,2004-09-05_1


In [113]:
#Date spliting 
#Weather conditions (such as rainy season/dry season) vary by month, affecting the horses' performance
def date_split(df):
    df["Date"] = pd.to_datetime(df["Date"])
    df["Month"] = df["Date"].dt.month
    return df

train_df = date_split(train_df)
test_df = date_split(test_df)
exam_df = date_split(exam_df)

In [114]:
def add_trainer_win_rate(train_df, test_df, exam_df):
    
    trainer_stats = train_df.groupby('Trainer')['Rank'].agg(
        win_count=lambda x: (x == 1).sum(),  # number of wins
        total_count='count'                  # total attending number
    )
    
    trainer_stats['win_rate'] = trainer_stats['win_count'] / (trainer_stats['total_count'] + 1e-6)
    
    global_win_rate = (train_df['Rank'] == 1).sum() / len(train_df)
    
   
    for df in [train_df, test_df, exam_df]:
        df['Trainer_Win_Rate'] = df['Trainer'].map(trainer_stats['win_rate']).fillna(global_win_rate)
    return train_df, test_df, exam_df

train_df, test_df, exam_df = add_trainer_win_rate(train_df, test_df, exam_df)


In [115]:
def get_feature_types(df):
    object_cols = [col for col in df.columns if df[col].dtype == 'object']
    categorical_cols = ["Session", "Ncourse", "RClass", "Runners", "Month"]
    indicator_cols = ["NewH", "HV", "AWT", "ST", "TrumpCard4", "NonGroupRace", "Rclass5"]
    Qualitative_cols = list(set(object_cols + categorical_cols + indicator_cols))
    
    numerical_cols = [col for col in df.columns if col not in Qualitative_cols]
    
    exclude_non_feature = ["Date", "Raceno", "HorseNo", "Race_Horse_ID", "Rank", "Race_ID"]
    
    Qualitative_cols = [c for c in Qualitative_cols if c not in exclude_non_feature]
    numerical_cols = [col for col in numerical_cols if col not in exclude_non_feature]
    
    return Qualitative_cols, numerical_cols, object_cols

Qualitative_cols, numerical_cols,object_cols = get_feature_types(train_df)

print(Qualitative_cols)
print(object_cols)
print(numerical_cols)

['ST', 'Trainer', 'NewH', 'Month', 'HV', 'TrumpCard4', 'RClass', 'Rclass5', 'Runners', 'Ncourse', 'NonGroupRace', 'AWT', 'Session']
['Trainer', 'Race_Horse_ID', 'Race_ID']
['NoRace', 'Going', 'Dist', 'RaceNoT', 'DistT', 'Drawing', 'HAgeNC', 'LHNR', 'LHNR20', 'SqrtLHNR20', 'HWgtNC', 'HWgtChgNC', 'ClaimNC', 'DrawC', 'StdDraw', 'DaySince', 'LastPubLogPP', 'Rating', 'WtCarried', 'LastLogOdds', 'HWinPer', 'HPPNC', 'NHPP', 'JWinPer', 'JPP', 'TWinPer', 'TPP', 'HAge', 'LifeHNoRace20', 'Brace2race', 'AveStdRank60', 'JAveStdRank60', 'TAveStdRank30', 'AveSpeedRating', 'LastSpeedRating', 'WtCarriedChg', 'DistTChg', 'LWP60', 'Trainer_Win_Rate']


In [116]:
#Focusing on the difference of horse in the same race
def normalize_race_features(train_df, test_df, exam_df, numerical_cols):
   
    scaler = RobustScaler()
    for col in numerical_cols:
        
        train_df[f'{col}_Norm'] = train_df.groupby('Race_ID')[col].transform(
            lambda x: scaler.fit_transform(x.values.reshape(-1,1)).flatten()
        )
        
        test_df[f'{col}_Norm'] = test_df.groupby('Race_ID')[col].transform(
            lambda x: scaler.transform(x.values.reshape(-1,1)).flatten()
        )
        exam_df[f'{col}_Norm'] = exam_df.groupby('Race_ID')[col].transform(
            lambda x: scaler.transform(x.values.reshape(-1,1)).flatten()
        )
    return train_df, test_df, exam_df

train_df, test_df, exam_df = normalize_race_features(train_df, test_df, exam_df, numerical_cols)

In [117]:
# Missing qualitative value handling -- Mode in training dataset
# Missing numerical value can be handled by xgboost automatically
for col in Qualitative_cols:
    mode = train_df[col].mode()[0]
    train_df[col] = train_df[col].fillna(mode)
    test_df[col] = test_df[col].fillna(mode)
    exam_df[col] = exam_df[col].fillna(mode)
    
for col in numerical_cols:
    median_val = train_df[col].median()
    train_df[col] = train_df[col].fillna(median_val)
    test_df[col] = test_df[col].fillna(median_val)
    exam_df[col] = exam_df[col].fillna(median_val)

In [118]:
#The problem of data sparsity:Out of the 202 trainers, 
#93 (46%) have only one record.121 trainers (60%) have less than 5 records.126 trainers (62%) have fewer than 10 records.
#This means that: The majority of the data for Trainers is very limited.It may be difficult to establish a reliable statistical model by trainer. 
#The winning rates of most Trainers range from 5% to 10%, but there are significant variations.
#Classify those attending freqency < 10 as "other"
if "Trainer" in Qualitative_cols:
    trainer_counts = train_df['Trainer'].value_counts()
    low_freq_trainers = trainer_counts[trainer_counts < 10].index.tolist()
    
    for df in [train_df, test_df, exam_df]:
        df['Trainer'] = df['Trainer'].apply(lambda x: 'Other' if x in low_freq_trainers else x)

In [119]:
# LabelEncoder for qualitative cols
label_encoders = {}
for col in Qualitative_cols:
    le = LabelEncoder()
    train_df[col] = le.fit_transform(train_df[col].astype(str))
    # Unseen category, encoded as -1
    test_df[col] = test_df[col].astype(str).map(lambda x: le.transform([x])[0] if x in le.classes_ else -1)
    exam_df[col] = exam_df[col].astype(str).map(lambda x: le.transform([x])[0] if x in le.classes_ else -1)
    label_encoders[col] = le

In [120]:
exclude_cols = ["Date", "Raceno", "HorseNo", "Race_Horse_ID", "Rank", "Race_ID"]
feature_cols = (
    Qualitative_cols + 
    [f'{col}_Norm' for col in numerical_cols]
)
print(feature_cols)

['ST', 'Trainer', 'NewH', 'Month', 'HV', 'TrumpCard4', 'RClass', 'Rclass5', 'Runners', 'Ncourse', 'NonGroupRace', 'AWT', 'Session', 'NoRace_Norm', 'Going_Norm', 'Dist_Norm', 'RaceNoT_Norm', 'DistT_Norm', 'Drawing_Norm', 'HAgeNC_Norm', 'LHNR_Norm', 'LHNR20_Norm', 'SqrtLHNR20_Norm', 'HWgtNC_Norm', 'HWgtChgNC_Norm', 'ClaimNC_Norm', 'DrawC_Norm', 'StdDraw_Norm', 'DaySince_Norm', 'LastPubLogPP_Norm', 'Rating_Norm', 'WtCarried_Norm', 'LastLogOdds_Norm', 'HWinPer_Norm', 'HPPNC_Norm', 'NHPP_Norm', 'JWinPer_Norm', 'JPP_Norm', 'TWinPer_Norm', 'TPP_Norm', 'HAge_Norm', 'LifeHNoRace20_Norm', 'Brace2race_Norm', 'AveStdRank60_Norm', 'JAveStdRank60_Norm', 'TAveStdRank30_Norm', 'AveSpeedRating_Norm', 'LastSpeedRating_Norm', 'WtCarriedChg_Norm', 'DistTChg_Norm', 'LWP60_Norm', 'Trainer_Win_Rate_Norm']


In [121]:
X_train = train_df[feature_cols].copy()
y_train = (train_df["Rank"] == 1).astype(int)

X_test = test_df[feature_cols].copy()
y_test = (test_df["Rank"] == 1).astype(int)

X_exam = exam_df[feature_cols].copy()

In [122]:
dtrain = xgb.DMatrix(X_train, label = y_train)
dtest = xgb.DMatrix(X_test, label = y_test)

In [123]:
params = {
    "objective": "binary:logistic",  
    "eval_metric": "auc",            
    "learning_rate": 0.1,            
    "max_depth": 5,                 
    "random_state": 42
}

# Early Stop Callback Function
early_stopping = xgb.callback.EarlyStopping(
    rounds=50,          # Stop after 50 cycles without imporovement
    metric_name="auc",  # Monitoring AUC
    save_best=True,    
)

In [124]:
xgb_model = xgb.train(
    params,
    dtrain,
    num_boost_round=1000,
    evals=[(dtest, "validation")],
    callbacks=[early_stopping],
    verbose_eval=10
)

[0]	validation-auc:0.70800
[10]	validation-auc:0.73489
[20]	validation-auc:0.74179
[30]	validation-auc:0.74365
[40]	validation-auc:0.74457
[50]	validation-auc:0.74472
[60]	validation-auc:0.74418
[70]	validation-auc:0.74210
[80]	validation-auc:0.74228
[90]	validation-auc:0.74179
[100]	validation-auc:0.74153
[102]	validation-auc:0.74141


In [125]:
y_pred_prob = xgb_model.predict(dtest)
y_pred = (y_pred_prob >= 0.5).astype(np.int64)

print("Test Accuracy：", accuracy_score(y_test, y_pred))
print("Classification Report：")
print(classification_report(y_test, y_pred))

Test Accuracy： 0.9190189237762977
Classification Report：
              precision    recall  f1-score   support

           0       0.92      1.00      0.96      8695
           1       0.42      0.01      0.01       764

    accuracy                           0.92      9459
   macro avg       0.67      0.50      0.49      9459
weighted avg       0.88      0.92      0.88      9459



In [141]:
# model optimization
xgb_model = xgb.XGBClassifier(
    objective="binary:logistic", 
    eval_metric="auc",            
    random_state=42,
    scale_pos_weight=len(y_train[y_train==0])/len(y_train[y_train==1]) #Highly unbiased
)

param_grid = { 
#     "learning_rate" : [0.03, 0.06, 0.1],
#     "max_depth" : [3, 5, 7],
#     "min_child_weight": [1, 3, 5],
#     "subsample" : [0.7, 0.9],  #Sampling ratio per tree
#     "colsample_bytree" : [0.65, 0.7, 0.75], #Percentage of Feature Samples per Tree
#     "reg_alpha": [0, 0.3, 0.6],
#     "reg_lambda": [1, 2, 3],
#     "gamma" : [0, 0.1, 0.3]
    "learning_rate": [0.06, 0.1],       # 核心：学习率（控制步长）
    "max_depth": [3, 5],               # 核心：树深度（控制复杂度）
    "min_child_weight": [1, 3],        # 次要：叶子节点最小权重（抑制过拟合）
    "subsample": [0.8],                # 固定：样本采样率（取中间值）
    "colsample_bytree": [0.7],         # 固定：特征采样率（取中间值）
    "reg_alpha": [0.3],                # 固定：L1正则（取中间值）
    "reg_lambda": [2],                 # 固定：L2正则（取中间值）
    "gamma": [0.1] 
}

# param_distributions = {
#     "learning_rate": [0.02, 0.03, 0.05, 0.07, 0.1, 0.15],
#     "max_depth": [3, 4, 5, 6, 7, 8],
#     "min_child_weight": [1, 2, 3, 4, 5, 6],
#     "subsample": [0.6, 0.7, 0.8, 0.9, 1.0],
#     "colsample_bytree": [0.5, 0.6, 0.7, 0.8, 0.9],
#     "reg_alpha": [0, 0.1, 0.3, 0.5, 1.0],
#     "reg_lambda": [0.5, 1, 2, 3, 5],
#     "gamma": [0, 0.05, 0.1, 0.2, 0.3],
#     "n_estimators": [200, 300, 500, 800, 1000]
# }

# 新增：分层KFold（适配不平衡数据）
cv_strategy = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

grid_search = GridSearchCV(
    estimator = xgb_model,
    param_grid = param_grid,
    cv = cv_strategy,
    scoring = 'f1_macro', #“won” horses was far fewer than that “did not win”--Unbalanced data
    n_jobs = -1,
    verbose = 2
)

grid_search.fit(X_train, y_train)

# random_search = RandomizedSearchCV(
#     estimator=xgb_model,
#     param_distributions=param_distributions,
#     n_iter=70,              # 推荐：50–100
#     scoring="f1_macro",
#     cv=cv_strategy,
#     verbose=2,
#     random_state=42,
#     n_jobs=-1
# )

# random_search.fit(X_train, y_train)

print('best parameter:', grid_search.best_params_)
print('best F1-score:', grid_search.best_score_.round(4))

Fitting 5 folds for each of 8 candidates, totalling 40 fits
best parameter: {'colsample_bytree': 0.7, 'gamma': 0.1, 'learning_rate': 0.1, 'max_depth': 5, 'min_child_weight': 1, 'reg_alpha': 0.3, 'reg_lambda': 2, 'subsample': 0.8}
best F1-score: 0.5396


In [142]:
best_xgb_model = grid_search.best_estimator_
y_pred = best_xgb_model.predict(X_test)

In [143]:
test_df['Win_Prob'] = best_xgb_model.predict_proba(X_test)[:, 1]  
test_df['Actual_Champion'] = (test_df['Rank'] == 1).astype(int)   

total_races = test_df['Race_ID'].nunique()

race_pred = test_df.groupby('Race_ID', group_keys=False).apply(
    lambda x: x.loc[x['Win_Prob'].idxmax()]  # choose the horse with highest probability to win
).reset_index(drop=True)

race_pred = race_pred[['Race_ID', 'HorseNo']].rename(columns={'HorseNo': 'Pred_Champion_Horse'})

race_actual = test_df[test_df['Actual_Champion'] == 1][['Race_ID', 'HorseNo']].rename(
    columns={'HorseNo': 'Actual_Champion_Horse'}
)

race_eval = pd.merge(
    race_pred,
    race_actual,
    on='Race_ID',
    how='inner'  
)

race_eval['Is_Correct'] = (race_eval['Pred_Champion_Horse'] == race_eval['Actual_Champion_Horse']).astype(int)

correct_races = race_eval['Is_Correct'].sum()
race_accuracy = (correct_races / total_races) * 100

print(f"True prediction number：{correct_races}")
print(f"Total number：{total_races}")
print(f"Prediction Accuracy on Races：{race_accuracy:.2f}%")


True prediction number：198
Total number：761
Prediction Accuracy on Races：26.02%


C:\Windows\Temp\ipykernel_6508\25388943.py:6: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  race_pred = test_df.groupby('Race_ID', group_keys=False).apply(


In [144]:
#Use train and test data to predict exam data
full_train_df = pd.concat([train_df, test_df], axis=0, ignore_index=True)

X_full_train = full_train_df[feature_cols].copy()
y_full_train = (full_train_df["Rank"] == 1).astype(int)


best_params = random_search.best_params_
final_xgb_model = xgb.XGBClassifier(
    objective="binary:logistic",
    eval_metric="auc",
    random_state=42,
    scale_pos_weight=len(y_full_train[y_full_train==0])/len(y_full_train[y_full_train==1]),
    **best_params  
)

final_xgb_model.fit(X_full_train, y_full_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.7, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='auc', feature_types=None,
              gamma=0, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.02, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=8, max_leaves=None,
              min_child_weight=3, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=800, n_jobs=None,
              num_parallel_tree=None, random_state=42, ...)

In [145]:
exam_df['Win_Prob'] = final_xgb_model.predict_proba(X_exam)[:, 1]

exam_race_pred = exam_df.groupby('Race_ID', group_keys=False).apply(
    lambda x: x.loc[x['Win_Prob'].idxmax()]
).reset_index(drop=True)

exam_race_pred = exam_race_pred[['Race_ID', 'HorseNo', 'Win_Prob']].rename(
    columns={
        'HorseNo': 'Pred_Champion_Horse',
        'Win_Prob': 'Champion_Win_Prob'
    }
)

exam_df = pd.merge(
    exam_df,
    exam_race_pred,
    on='Race_ID',
    how='left'
)
# 标记是否为预测冠军
exam_df['Is_Pred_Champion'] = (exam_df['HorseNo'] == exam_df['Pred_Champion_Horse']).astype(int)

# 6. 将预测结果写入Excel的Exam工作表
print("\n=== 写入预测结果到Excel ===")
# 需安装openpyxl：pip install openpyxl
with pd.ExcelWriter('Horse-Data_student.xlsx', engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
    exam_df.to_excel(writer, sheet_name='Exam', index=False)

# 7. 输出Exam预测结果摘要
print("\n=== Exam数据预测结果摘要 ===")
print(f"Exam数据总行数：{len(exam_df)}")
print(f"Exam赛事总数：{exam_df['Race_ID'].nunique()}")
print(f"预测冠军总数：{exam_df['Is_Pred_Champion'].sum()}")
print("\n前10场赛事的预测冠军：")
print(exam_race_pred.head(10))

C:\Windows\Temp\ipykernel_6508\2112311890.py:3: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  exam_race_pred = exam_df.groupby('Race_ID', group_keys=False).apply(



=== 写入预测结果到Excel ===

=== Exam数据预测结果摘要 ===
Exam数据总行数：9655
Exam赛事总数：763
预测冠军总数：763

前10场赛事的预测冠军：
         Race_ID  Pred_Champion_Horse  Champion_Win_Prob
0   2011-09-11_1                   14           0.599097
1  2011-09-11_10                   10           0.631236
2   2011-09-11_2                    7           0.354652
3   2011-09-11_3                    4           0.520357
4   2011-09-11_4                    4           0.592640
5   2011-09-11_5                    1           0.561767
6   2011-09-11_6                    8           0.648699
7   2011-09-11_7                   10           0.756365
8   2011-09-11_8                    6           0.779323
9   2011-09-11_9                    1           0.590640
